In [1]:
import numpy as np
import pickle

# 加载数据
with open('R_ET_1_ET_2_ET_3_0.pkl', 'rb') as file:
    data = pickle.load(file)

# 提取 9x9 的 Fisher 矩阵，并仅保留前 7 个参数的子矩阵
def extract_relevant_submatrix(matrix, size=7):
    """提取前 size 个参数的子矩阵"""
    return matrix[:size, :size]

fisher_matrix_1 = extract_relevant_submatrix(data[1])
fisher_matrix_2 = extract_relevant_submatrix(data[2][0])
fisher_matrix_3 = extract_relevant_submatrix(data[2][1])

In [2]:
# 动态正则化项函数
def get_dynamic_epsilon(matrix):
    cond_number = np.linalg.cond(matrix)
    eigvals = np.linalg.eigvals(matrix)
    eigval_std = np.std(eigvals)
    
    if cond_number > 1e20 or eigval_std > 1e2:
        return 1e-2
    elif cond_number > 1e16 or eigval_std > 1e1:
        return 1e-3
    elif cond_number > 1e12:
        return 1e-4
    else:
        return 1e-6

# 确保矩阵正定
def ensure_positive_definite(matrix, matrix_name=""):
    eigvals, eigvecs = np.linalg.eigh(matrix)
    print(f"{matrix_name} Eigenvalues before clipping: {eigvals}")
    eigvals_clipped = np.clip(eigvals, 1e-2, 1e3)
    print(f"{matrix_name} Eigenvalues after clipping: {eigvals_clipped}")
    matrix_pos_def = eigvecs @ np.diag(eigvals_clipped) @ eigvecs.T
    return matrix_pos_def

In [3]:
# 正则化矩阵
def regularize_matrix(matrix, matrix_name=""):
    epsilon = get_dynamic_epsilon(matrix)
    matrix_sym = (matrix + matrix.T) / 2
    return ensure_positive_definite(matrix_sym + epsilon * np.eye(matrix.shape[0]), matrix_name)

# 计算协方差矩阵
def compute_covariance(fisher_matrix, matrix_name=""):
    reg_matrix = regularize_matrix(fisher_matrix, matrix_name)
    cov_matrix = np.linalg.pinv(reg_matrix)
    return cov_matrix

In [4]:
# 计算协方差矩阵
cov_matrix_1 = compute_covariance(fisher_matrix_1, "matrix_1")
cov_matrix_2 = compute_covariance(fisher_matrix_2, "matrix_2")
cov_matrix_3 = compute_covariance(fisher_matrix_3, "matrix_3")

# 打印验证
print("Shape of fisher_matrix_1:", fisher_matrix_1.shape)
print("Shape of cov_matrix_1:", cov_matrix_1.shape)
print("Shape of fisher_matrix_2:", fisher_matrix_2.shape)
print("Shape of cov_matrix_2:", cov_matrix_2.shape)
print("Shape of fisher_matrix_3:", fisher_matrix_3.shape)
print("Shape of cov_matrix_3:", cov_matrix_3.shape)

matrix_1 Eigenvalues before clipping: [1.29689868e+00 5.87254906e+02 1.52861020e+05 6.27371238e+05
 2.98962960e+06 7.27654712e+10 2.74498770e+13]
matrix_1 Eigenvalues after clipping: [   1.29689868  587.25490559 1000.         1000.         1000.
 1000.         1000.        ]
matrix_2 Eigenvalues before clipping: [1.01472676e-02 1.14831407e-02 2.85245417e+04 1.17082957e+05
 4.62740919e+05 2.68476583e+07 2.14312886e+13]
matrix_2 Eigenvalues after clipping: [1.01472676e-02 1.14831407e-02 1.00000000e+03 1.00000000e+03
 1.00000000e+03 1.00000000e+03 1.00000000e+03]
matrix_3 Eigenvalues before clipping: [1.29689868e+00 5.87254906e+02 1.52861020e+05 6.27371238e+05
 2.98962960e+06 7.27654712e+10 2.74498770e+13]
matrix_3 Eigenvalues after clipping: [   1.29689868  587.25490559 1000.         1000.         1000.
 1000.         1000.        ]
Shape of fisher_matrix_1: (7, 7)
Shape of cov_matrix_1: (7, 7)
Shape of fisher_matrix_2: (7, 7)
Shape of cov_matrix_2: (7, 7)
Shape of fisher_matrix_3: (7, 7

In [5]:
# 优化均值计算
mean1 = np.mean(cov_matrix_1, axis=0)
mean2 = np.mean(cov_matrix_2, axis=0)
mean3 = np.mean(cov_matrix_3, axis=0)

# 动态调节 KL 散度中的 Trace 和均值差项
def dynamic_clip(value, threshold=250):
    return min(value, threshold)

In [6]:
# 计算 KL 散度
def kl_divergence(cov1, mean1, cov2, mean2, matrix_name=""):
    cov2_regularized = regularize_matrix(cov2, matrix_name)
    eigvals_cov2 = np.linalg.eigvals(cov2_regularized)
    if np.any(eigvals_cov2 <= 0):
        print(f"{matrix_name} has non-positive eigenvalues, skipping KL computation.")
        return np.nan

    try:
        inv_cov2 = np.linalg.inv(cov2_regularized)
    except np.linalg.LinAlgError:
        print(f"{matrix_name} inversion failed, skipping KL computation.")
        return np.nan

    term1 = np.trace(inv_cov2 @ cov1)
    term1 = dynamic_clip(term1)
    print(f"{matrix_name} KL term1 (Trace): {term1}")

    sign1, logdet1 = np.linalg.slogdet(cov1)
    sign2, logdet2 = np.linalg.slogdet(cov2_regularized)
    if sign1 <= 0 or sign2 <= 0:
        print(f"{matrix_name} determinant is non-positive.")
        return np.nan

    term2 = logdet2 - logdet1
    print(f"{matrix_name} KL term2 (Log determinant difference): {term2}")

    term3 = (mean2 - mean1).T @ inv_cov2 @ (mean2 - mean1)
    term3 = dynamic_clip(term3)
    print(f"{matrix_name} KL term3 (Mean difference): {term3}")

    result = 0.5 * (term1 + term2 + term3 - cov1.shape[0])
    print(f"{matrix_name} KL result: {result}")

    return max(result, 0)

In [7]:
# 打印 KL 散度
kl_12 = kl_divergence(cov_matrix_1, mean1, cov_matrix_2, mean2, "matrix_12")
kl_13 = kl_divergence(cov_matrix_1, mean1, cov_matrix_3, mean3, "matrix_13")
kl_23 = kl_divergence(cov_matrix_2, mean2, cov_matrix_3, mean3, "matrix_23")

print("\n==== KL Divergence ====")
print(f"KL Divergence between matrix 1 and matrix 2: {kl_12}")
print(f"KL Divergence between matrix 1 and matrix 3: {kl_13}")
print(f"KL Divergence between matrix 2 and matrix 3: {kl_23}")

matrix_12 Eigenvalues before clipping: [2.00000000e-03 2.00000000e-03 2.00000000e-03 2.00000000e-03
 2.00000000e-03 8.70851893e+01 9.85496973e+01]
matrix_12 Eigenvalues after clipping: [1.00000000e-02 1.00000000e-02 1.00000000e-02 1.00000000e-02
 1.00000000e-02 8.70851893e+01 9.85496973e+01]
matrix_12 KL term1 (Trace): 0.5079985941702951
matrix_12 KL term2 (Log determinant difference): 27.20580801480043
matrix_12 KL term3 (Mean difference): 1.1739928746981054
matrix_12 KL result: 10.943899741834414
matrix_13 Eigenvalues before clipping: [0.001001   0.001001   0.001001   0.001001   0.001001   0.00170384
 0.77107126]
matrix_13 Eigenvalues after clipping: [0.01       0.01       0.01       0.01       0.01       0.01
 0.77107126]
matrix_13 KL term1 (Trace): 1.6702825083950357
matrix_13 KL term2 (Log determinant difference): 13.283215552876431
matrix_13 KL term3 (Mean difference): 0.0
matrix_13 KL result: 3.976749030635734
matrix_23 Eigenvalues before clipping: [0.001001   0.001001   0.00100

In [8]:
def frobenius_distance(matrix1, matrix2):
    """计算两个矩阵之间的 Frobenius 距离"""
    return np.linalg.norm(matrix1 - matrix2, 'fro')

def print_frobenius_results(dist_12, dist_13, dist_23):
    """格式化输出 Frobenius 距离结果"""
    print("\n==== Frobenius Distances ====")
    print(f"Distance between matrix 1 and matrix 2: {dist_12:.4f}")
    print(f"Distance between matrix 1 and matrix 3: {dist_13:.4f}")
    print(f"Distance between matrix 2 and matrix 3: {dist_23:.4f}")

# 计算并输出 Frobenius 距离
dist_12 = frobenius_distance(cov_matrix_1, cov_matrix_2)
dist_13 = frobenius_distance(cov_matrix_1, cov_matrix_3)
dist_23 = frobenius_distance(cov_matrix_2, cov_matrix_3)

print_frobenius_results(dist_12, dist_13, dist_23)


==== Frobenius Distances ====
Distance between matrix 1 and matrix 2: 130.9351
Distance between matrix 1 and matrix 3: 0.0000
Distance between matrix 2 and matrix 3: 130.9351
